In [2]:
import pandas as pd
import sklearn
import datetime
import time

In [3]:
# load data
train_identity = pd.read_csv(r'.\ieee-fraud-detection\train_identity.csv')
train_transaction = pd.read_csv(r'.\ieee-fraud-detection\train_transaction.csv')
test_identity = pd.read_csv(r'.\ieee-fraud-detection\test_identity.csv')
test_transaction = pd.read_csv(r'.\ieee-fraud-detection\test_transaction.csv')

In [4]:
# left join on TransactionID, store in train
train = pd.merge(train_transaction, train_identity, on = 'TransactionID', how = 'left')
test = pd.merge(test_transaction, test_identity, on = 'TransactionID', how = 'left')
train_label = train['isFraud']
train = train.drop(columns = ['isFraud'])

In [5]:
all_data = pd.concat([train, test], axis = 0)

In [389]:
def dataloader():
    train_identity = pd.read_csv(r'.\ieee-fraud-detection\train_identity.csv')
    train_transaction = pd.read_csv(r'.\ieee-fraud-detection\train_transaction.csv')
    test_identity = pd.read_csv(r'.\ieee-fraud-detection\test_identity.csv')
    test_transaction = pd.read_csv(r'.\ieee-fraud-detection\test_transaction.csv')
    train = pd.merge(train_transaction, train_identity, on = 'TransactionID', how = 'left')
    test = pd.merge(test_transaction, test_identity, on = 'TransactionID', how = 'left')
    train_label = train['isFraud']
    train = train.drop(columns = ['isFraud'])
    all_data = pd.concat([train, test], axis = 0)
    return all_data, train, test, train_label

In [7]:
def get_feature_type(df):
    # make a dictionary to store all feature type
    # we know int and float are continuous O is categorical
    # cont for continuous
    # cat for catagorical
    dtype_dict = {}
    for i in df.columns.tolist():
        dtype_dict[i]= train[i].dtype
    
    feature_type = {}
    for i in dtype_dict.keys():
        if dtype_dict[i] == 'int64':
            feature_type[i] = 'cont'
        if dtype_dict[i] == 'float64':
            feature_type[i] = 'cont'
        if dtype_dict[i] == 'O':
            feature_type[i] = 'cat'
    return feature_type

In [32]:
def find_unique_value(df):
    feature_type = get_feature_type(df)
    feature_unique_value = {}
    for i in df.columns.tolist():
        if feature_type[i] == 'cat':
            feature_unique_value[i]=len(set(all_data[i].unique()))
    return feature_unique_value

def reduce_f_dim(col, critical_value):
    temp = col.value_counts().to_dict()
    total = sum(temp.values())
    new_col = []
    for keys in col:
        if temp[keys] / total > critical_value:
            new_col.append(keys)
        else:
            new_col.append('etc')
    return new_col

def categorical_feature_reduce_dim(df, critical_feature_num, critical_value):
    feature_unique_value = find_unique_value(all_data)
    for i in feature_unique_value.keys():
        if feature_unique_value[i] > critical_feature_num:
            col = df[i]
            fill_value = 'null'
            col = col.fillna(fill_value)
            col = reduce_f_dim(col, critical_value)
            df[i] = col
    return df
temp = categorical_feature_reduce_dim(train, 8, 0.05)

In [9]:
def add_date(df):
    startdate = datetime.datetime.strptime('20190101', '%Y%m%d')
    df['Date'] = df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
    df['Week'] = df['Date'].dt.dayofweek
    df['days'] = df['Date'].dt.day
    df['hours'] = df['Date'].dt.hour
    df = df.drop(columns=['Date'])
    return df

In [24]:
# find percentage of non null value of each feature
def find_null_percentage(df):
    null = {}
    for i in df.columns.tolist():
        null[i] = 1 - df[i].isnull().value_counts()[False]/df.shape[0]
    return null

def fill_na_numeric(df, critical_value):
    original_dim = df.shape[1]
    null = find_null_percentage(all_data)
    columns_to_drop = []
    for i in null.keys():
        if null[i] > critical_value:
            columns_to_drop.append(i)
    df = df.drop(columns=columns_to_drop)

    return df

In [33]:
temp = add_date(temp)
print(temp.shape)
temp = fill_na_numeric(temp, 0.25)


(590540, 436)


In [34]:
temp.shape

(590540, 203)

In [22]:
null = find_null_percentage(all_data)

In [35]:
temp.columns.tolist()

['TransactionID',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'P_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D4',
 'D10',
 'D15',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V77',
 'V78',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V86',
 'V87',
 'V88',
 'V89',
 'V90',
 'V91',
 'V92',
 'V93',
 'V94',
 'V95',
 'V96',
 'V97',
 'V98',
 'V9

In [388]:
len([i for i in null.values() if i < 0.25])

200

In [323]:
find_unique_value(all_data)

{'ProductCD': 5,
 'card4': 5,
 'card6': 5,
 'P_emaildomain': 61,
 'R_emaildomain': 61,
 'M1': 3,
 'M2': 3,
 'M3': 3,
 'M4': 4,
 'M5': 3,
 'M6': 3,
 'M7': 3,
 'M8': 3,
 'M9': 3,
 'id_12': 3,
 'id_15': 4,
 'id_16': 3,
 'id_23': 4,
 'id_27': 3,
 'id_28': 3,
 'id_29': 3,
 'id_30': 88,
 'id_31': 173,
 'id_33': 462,
 'id_34': 5,
 'id_35': 3,
 'id_36': 3,
 'id_37': 3,
 'id_38': 3,
 'DeviceType': 3,
 'DeviceInfo': 2800}

In [351]:
all_data['dist1']

0          19.0
1           NaN
2         287.0
3           NaN
4           NaN
5          36.0
6           0.0
7           NaN
8           NaN
9          19.0
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15          3.0
16          NaN
17          NaN
18          5.0
19          NaN
20          0.0
21          NaN
22          NaN
23          4.0
24          NaN
25          NaN
26          NaN
27         17.0
28          NaN
29          NaN
          ...  
506661      NaN
506662      NaN
506663     66.0
506664    153.0
506665      NaN
506666      NaN
506667      9.0
506668      8.0
506669     13.0
506670      NaN
506671      NaN
506672      NaN
506673     11.0
506674      NaN
506675      NaN
506676      NaN
506677      NaN
506678      NaN
506679      NaN
506680      NaN
506681      NaN
506682      9.0
506683      0.0
506684      NaN
506685      NaN
506686      NaN
506687      NaN
506688      NaN
506689      NaN
506690      NaN
Name: dist1, Length: 109